# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pprint import pprint

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [27]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("../../data/train_data_0817.csv")
test_data = pd.read_csv("../../data/test_data_0817.csv")

In [28]:
train_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Data columns (total 38 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   CURE SPEED Collect Result_Dam                   40506 non-null  int64  
 1   DISCHARGED SPEED OF RESIN Collect Result_Dam    40506 non-null  int64  
 2   Head Clean Position Z Collect Result_Dam        40506 non-null  float64
 3   Head Purge Position Z Collect Result_Dam        40506 non-null  float64
 4   Head Zero Position Y Collect Result_Dam         40506 non-null  float64
 5   Stage2_Circle_Distance_Speed_Dam                40506 non-null  int64  
 6   WorkMode Collect Result                         40506 non-null  float64
 7   Chamber Temp. Collect Result_AutoClave          40506 non-null  int64  
 8   DISCHARGED SPEED OF RESIN Collect Result_Fill1  40506 non-null  float64
 9   Head Purge Position Z Collect Result_Fi

In [29]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [30]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### train
var_all_train = [
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

In [31]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [32]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [33]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [34]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [35]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


---

## 모델링

### 모델 정의

In [37]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name]

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

# 사용 예시
# train_and_evaluate_model(
#     'lgbm', train_data_fill1,
#     n_estimators=979,
#     num_leaves=1565,
#     max_depth=34,
#     learning_rate=0.04888906225539191,
#     min_child_samples=36,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )


def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

# 사용 예시
# model_dam = fit_all_train_data_function(
#     'lgbm', train_data_dam,
#     n_estimators=2748,
#     num_leaves=657,
#     max_depth=256,
#     learning_rate=0.001043279508273329,
#     min_child_samples=58,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )


### 모델 학습

Dam 모델

In [38]:
train_and_evaluate_model(
    'lgbm', train_data_dam
    , n_estimators=1901
    , num_leaves=1247
    , max_depth=36
    , learning_rate=0.0011375205361216431 
    , min_child_samples=129
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.21949078138718175
---
Confusion Matrix:
[[7088  574]
 [ 315  125]]
---
Accuracy: 0.8902740064181683
Precision: 0.17882689556509299
Recall: 0.2840909090909091




AutoClave 모델

In [39]:
train_and_evaluate_model(
    'lgbm', train_data_autoclave
    , n_estimators=2733
    , num_leaves=2080
    , max_depth=17
    , learning_rate=0.048548896042732353 
    , min_child_samples=206
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.2374821173104435
---
Confusion Matrix:
[[7486  176]
 [ 357   83]]
---
Accuracy: 0.9342137743766971
Precision: 0.3204633204633205
Recall: 0.18863636363636363




Fill1 모델

In [40]:
train_and_evaluate_model(
    'lgbm', train_data_fill1
    , n_estimators=1399
    , num_leaves=1456
    , max_depth=38
    , learning_rate=0.0020188481674757053 
    , min_child_samples=21
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.21714285714285714
---
Confusion Matrix:
[[7166  496]
 [ 326  114]]
---
Accuracy: 0.898543569489015
Precision: 0.18688524590163935
Recall: 0.2590909090909091




Fill2 모델

In [41]:
train_and_evaluate_model(
    'lgbm', train_data_fill2
    , n_estimators=1997
    , num_leaves=1641
    , max_depth=10
    , learning_rate=0.07435948250635764 
    , min_child_samples=114
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.2166405023547881
---
Confusion Matrix:
[[7534  128]
 [ 371   69]]
---
Accuracy: 0.9384102690693656
Precision: 0.350253807106599
Recall: 0.15681818181818183




---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [42]:
model_Dam = fit_all_train_data_function(
    'lgbm', train_data_dam
    , n_estimators=1901
    , num_leaves=1247
    , max_depth=36
    , learning_rate=0.0011375205361216431 
    , min_child_samples=129
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_AutoClave = fit_all_train_data_function(
    'lgbm', train_data_autoclave
    , n_estimators=2733
    , num_leaves=2080
    , max_depth=17
    , learning_rate=0.048548896042732353 
    , min_child_samples=206
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill1 = fit_all_train_data_function(
    'lgbm', train_data_fill1
    , n_estimators=1399
    , num_leaves=1456
    , max_depth=38
    , learning_rate=0.0020188481674757053 
    , min_child_samples=21
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill2 = fit_all_train_data_function(
    'lgbm', train_data_fill2
    , n_estimators=1997
    , num_leaves=1641
    , max_depth=10
    , learning_rate=0.07435948250635764 
    , min_child_samples=114
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_dam 데이터로 학습 완료
lgbm 모델이 train_data_autoclave 데이터로 학습 완료
lgbm 모델이 train_data_fill1 데이터로 학습 완료
lgbm 모델이 train_data_fill2 데이터로 학습 완료


---

In [48]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)

# 각 공정의 예측 확률 계산
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
]

In [49]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.3 적용
final_predictions = (soft_voting_probs >= 0.3).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

0


In [50]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.28 적용
final_predictions = (soft_voting_probs >= 0.28).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

0


In [51]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.28 적용
final_predictions = (soft_voting_probs >= 0.2).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

25


## 4. 제출하기


### 제출 파일 작성


In [23]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("../../data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [24]:
df_sub['target'].value_counts()

Normal      17336
AbNormal       25
Name: target, dtype: int64

In [25]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
